# Example of Sensitivity Analysis

In [ ]:
# Defaults
n_samples = 1024 * 21

In [ ]:
from itertools import repeat

import numpy as np
import pandas as pd

import dask.bag as db
from dask.distributed import Client
from dask_jobqueue import SLURMCluster as Cluster

from SALib.sample import latin
from SALib.analyze import delta as delta_mim

from mhpc_project.matsch_b2 import Variables, CalibrationModel, Loss
from geotopy.measures import KGE
from geotopy.utils import date_parser

In [ ]:
observations = pd.read_csv('../data/Matsch B2/obs.csv',
                           na_values=['-9999', '-99.99'],
                           usecols=[0, 7],
                           parse_dates=[0],
                           date_parser=date_parser,
                           index_col=0,
                           squeeze=True)
observations.index.rename('datetime', inplace=True)
model = CalibrationModel('../data/Matsch B2/geotop', run_args={'timeout': 120})
variables = Variables('../data/Matsch B2/variables.csv')
measure = KGE(observations)
loss = Loss(model, variables, measure)

In [ ]:
cluster = Cluster()
client = Client(cluster)

In [ ]:
problem = {'num_vars': variables.num_vars,
           'names': variables.names,
           'bounds': list(repeat((0.0, 1.0), variables.num_vars))}

samples = latin.sample(problem, n_samples)

In [ ]:
bag = db.from_sequence(samples, npartitions=512).map(loss)
losses = bag.compute()

In [ ]:
losses = np.array(losses)
indices = np.isfinite(losses)
good_samples = samples[indices]
good_losses = losses[indices]

SA = delta_mim.analyze(problem, good_samples, good_losses)

In [ ]:
SA.to_df().sort_values('delta', key=np.abs, ascending=False)